In [1]:
# Dependancies
import pandas as pd
import os
import csv

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

# Import modules to declare columns and column data types
from sqlalchemy import Column, Integer, String, Float

# Import and establish Base for which classes will be constructed 
from sqlalchemy.ext.declarative import declarative_base

# Define Declarative Base 
Base = declarative_base()

In [2]:
# Inspecting csv file
df=pd.read_csv('db/leads_updated.csv', dtype={'Zip': 'str'})
#file = "db/leads_updatednew.csv",dtype={'Zip':'str'}
#df = pd.read_csv(file)
df.head(1000)

,ConsumerID,Zip,latitude,longitude,Audience Count,City,State,Address,Country,Gender,...,Occupation,DwellingType,WorkingWoman,GreenLiving,CreditScore,#Kids,#Adults,AGEBIN,Household Income,Email Address
0,1,75253,32.67,-96.61,1,Dallas,TX,Texas,US,Female,...,Unknown,02. Multiple Family Dwelling Unit,02. Unknown,02. Unknown,02. Less Stable,Unknown,1 Adult,18-25,20. Unknown,xxx@gmail.com
1,2,19147,39.94,-75.15,1,Philadelphia,PA,Pennsylvania,US,Male,...,Unknown,01. Single Family Dwelling Unit,02. Unknown,02. Unknown,01. Least Stable,1 Kid,1 Adult,41-60,"05. $25,000 - $29,999",xxx@gmail.com
2,3,33903,26.68,-81.91,1,North Fort Myers,FL,Florida,US,Male,...,Unknown,01. Single Family Dwelling Unit,02. Unknown,01. True,01. Least Stable,Unknown,2 Adults,61+,"05. $25,000 - $29,999",xxx@gmail.com
3,4,35124,33.30,-86.79,1,Pelham,AL,Alabama,US,Male,...,Unknown,01. Single Family Dwelling Unit,02. Unknown,02. Unknown,03. Stable,1 Kid,3 Adults,18-25,"13. $65,000 - $74,999",xxx@gmail.com
4,5,11208,40.67,-73.87,1,Brooklyn,NY,New York,US,Female,...,Homemaker,01. Single Family Dwelling Unit,02. Unknown,02. Unknown,03. Stable,2 Kids,1 Adult,26-40,"02. $10,000 - $14,999",xxx@gmail.com
5,6,80224,39.69,-104.91,1,Denver,CO,Colorado,US,Female,...,Unknown,02. Multiple Family Dwelling Unit,02. Unknown,02. Unknown,01. Least Stable,Unknown,Unknown,18-25,20. Unknown,xxx@gmail.com
6,7,35805,34.70,-86.62,1,Huntsville,AL,Alabama,US,Female,...,Unknown,01. Single Family Dwelling Unit,02. Unknown,02. Unknown,01. Least Stable,Unknown,1 Adult,18-25,"03. $15,000 - $19,999",xxx@gmail.com
7,8,60060,42.26,-88.00,1,Mundelein,IL,Illinois,US,Male,...,Craftsman / Blue Collar,01. Single Family Dwelling Unit,02. Unknown,02. Unknown,04. High Stable,1 Kid,1 Adult,41-60,"13. $65,000 - $74,999",xxx@gmail.com
8,9,33174,25.76,-80.36,1,Miami,FL,Florida,US,Male,...,Clerical / White Collar,01. Single Family Dwelling Unit,02. Unknown,02. Unknown,01. Least Stable,1 Kid,1 Adult,41-60,"02. $10,000 - $14,999",xxx@gmail.com
9,10,33326,26.11,-80.39,1,Weston,FL,Florida,US,Female,...,Clerical / White Collar,02. Multiple Family Dwelling Unit,01. True,02. Unknown,02. Less Stable,Unknown,1 Adult,26-40,"14. $75,000 - $99,999",xxx@gmail.com


In [3]:
# Rename the columns so it is satisfies the requirements when creating the class in sqlite 
new_df = df.rename(index=str, columns={"#Kids": "Kids", "#Adults": "Adults", "Audience Count": "Audience_Count", "Household Income": "Household_Income", "Email Address": "Email_Address"})
new_df

,ConsumerID,Zip,latitude,longitude,Audience_Count,City,State,Address,Country,Gender,...,Occupation,DwellingType,WorkingWoman,GreenLiving,CreditScore,Kids,Adults,AGEBIN,Household_Income,Email_Address
0,1,75253,32.67,-96.61,1,Dallas,TX,Texas,US,Female,...,Unknown,02. Multiple Family Dwelling Unit,02. Unknown,02. Unknown,02. Less Stable,Unknown,1 Adult,18-25,20. Unknown,xxx@gmail.com
1,2,19147,39.94,-75.15,1,Philadelphia,PA,Pennsylvania,US,Male,...,Unknown,01. Single Family Dwelling Unit,02. Unknown,02. Unknown,01. Least Stable,1 Kid,1 Adult,41-60,"05. $25,000 - $29,999",xxx@gmail.com
2,3,33903,26.68,-81.91,1,North Fort Myers,FL,Florida,US,Male,...,Unknown,01. Single Family Dwelling Unit,02. Unknown,01. True,01. Least Stable,Unknown,2 Adults,61+,"05. $25,000 - $29,999",xxx@gmail.com
3,4,35124,33.30,-86.79,1,Pelham,AL,Alabama,US,Male,...,Unknown,01. Single Family Dwelling Unit,02. Unknown,02. Unknown,03. Stable,1 Kid,3 Adults,18-25,"13. $65,000 - $74,999",xxx@gmail.com
4,5,11208,40.67,-73.87,1,Brooklyn,NY,New York,US,Female,...,Homemaker,01. Single Family Dwelling Unit,02. Unknown,02. Unknown,03. Stable,2 Kids,1 Adult,26-40,"02. $10,000 - $14,999",xxx@gmail.com
5,6,80224,39.69,-104.91,1,Denver,CO,Colorado,US,Female,...,Unknown,02. Multiple Family Dwelling Unit,02. Unknown,02. Unknown,01. Least Stable,Unknown,Unknown,18-25,20. Unknown,xxx@gmail.com
6,7,35805,34.70,-86.62,1,Huntsville,AL,Alabama,US,Female,...,Unknown,01. Single Family Dwelling Unit,02. Unknown,02. Unknown,01. Least Stable,Unknown,1 Adult,18-25,"03. $15,000 - $19,999",xxx@gmail.com
7,8,60060,42.26,-88.00,1,Mundelein,IL,Illinois,US,Male,...,Craftsman / Blue Collar,01. Single Family Dwelling Unit,02. Unknown,02. Unknown,04. High Stable,1 Kid,1 Adult,41-60,"13. $65,000 - $74,999",xxx@gmail.com
8,9,33174,25.76,-80.36,1,Miami,FL,Florida,US,Male,...,Clerical / White Collar,01. Single Family Dwelling Unit,02. Unknown,02. Unknown,01. Least Stable,1 Kid,1 Adult,41-60,"02. $10,000 - $14,999",xxx@gmail.com
9,10,33326,26.11,-80.39,1,Weston,FL,Florida,US,Female,...,Clerical / White Collar,02. Multiple Family Dwelling Unit,01. True,02. Unknown,02. Less Stable,Unknown,1 Adult,26-40,"14. $75,000 - $99,999",xxx@gmail.com


In [4]:
# Listin all the columns
# list(new_df)

In [5]:
# # Establish the connection to sql DB
# rds_connection_string = f"root:{db_pass}@127.0.0.1/new_leads_db"
# engine = create_engine(f'mysql+pymysql://{rds_connection_string}')

In [6]:
# # Read in the Data from the Table 
# leads_df = pd.read_sql('SELECT * FROM leads_updated', con=engine)
# leads_df

In [7]:
# Creating an engine, sqlite database and connection to the engine
engine = create_engine("sqlite:///db/leads.sqlite", echo=False)
conn = engine.connect()

In [8]:
# Creating a Class & table 
class Leads(Base):
    __tablename__ = 'leads_table'
    id = Column(Integer, primary_key=True)
    ConsumerID = Column(Integer)
    Zip = Column(String(255))
    latitude = Column(Float)
    longitude = Column(Float)
    Audience_Count = Column(Integer)
    City = Column(String(255))
    State = Column(String(255))
    Address = Column(String(255))
    Country = Column(String(255))
    Gender = Column(String(255))
    Age = Column(Integer)
    MaritalStatus = Column(String(255))
    Education = Column(String(255))
    EthnicGroup = Column(String(255))
    EmploymentStatus = Column(String(255))
    HispanicIndicator = Column(String(255))
    email_ind = Column(Float)
    phn_ind = Column(Float)
    postal_ind = Column(Float)
    postal_ind = Column(Float)
    digital_ind = Column(Float)
    Occupation = Column(String(255))
    DwellingType = Column(String(255))
    WorkingWoman = Column(String(255))
    GreenLiving = Column(String(255))
    CreditScore = Column(String(255))
    Kids = Column(String(255))
    Adults = Column(String(255))
    AGEBIN = Column(String(255))
    Household_Income = Column(String(255))
    Email_Address = Column(String(255))

In [9]:
Base.metadata.create_all(conn)

In [10]:
# Transferring the data from dataframe to the sqlite db
new_df.to_sql(name='leads_table', con=conn, if_exists='append', index=False)

In [11]:
# Creating a session 
session = Session(bind=engine)

In [12]:
# Inspecting the Table to make sure the table is there
inspector = inspect(engine)
inspector.get_table_names()

['leads_table']

In [13]:
# Checking the data 
engine.execute('SELECT * FROM leads_table LIMIT 10').fetchall()

[(1, 1, 75253.0, 32.67, -96.61, 1, 'Dallas', 'TX', 'Texas', 'US', 'Female', -1, 'Single', 'Unknown', 'Hispanic', 'Unknown', 'Y', 1.0, 1.0, 1.0, 1.0, 'Unknown', '02. Multiple Family Dwelling Unit', '02. Unknown', '02. Unknown', '02. Less Stable', 'Unknown', '1 Adult', '18-25', '20. Unknown', 'xxx@gmail.com'),
 (2, 2, 19147.0, 39.94, -75.15, 1, 'Philadelphia', 'PA', 'Pennsylvania', 'US', 'Male', 60, 'Inferred Married', 'Unknown', 'Hispanic', 'Unknown', 'Y', 1.0, 0.0, 1.0, 1.0, 'Unknown', '01. Single Family Dwelling Unit', '02. Unknown', '02. Unknown', '01. Least Stable', '1 Kid', '1 Adult', '41-60', '05. $25,000 - $29,999', 'xxx@gmail.com'),
 (3, 3, 33903.0, 26.68, -81.91, 1, 'North Fort Myers', 'FL', 'Florida', 'US', 'Male', 68, 'Married', 'College', 'Hispanic', 'Sales / Service', 'Y', 1.0, 1.0, 1.0, 1.0, 'Unknown', '01. Single Family Dwelling Unit', '02. Unknown', '01. True', '01. Least Stable', 'Unknown', '2 Adults', '61+', '05. $25,000 - $29,999', 'xxx@gmail.com'),
 (4, 4, 35124.0, 3